# Scyclone-PyTorch
[![Generic badge](https://img.shields.io/badge/GitHub-Scyclone--PyTorch-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]
[![Paper](http://img.shields.io/badge/paper-arxiv.2005.03334-B31B1B.svg)][paper]  

Reimplmentation of voice conversion system "Scyclone" with PyTorch  
Author: [tarepan](https://github.com/tarepan)

[github]:https://github.com/tarepan/Scyclone-PyTorch
[paper]:https://arxiv.org/abs/2005.03334
[notebook]:https://colab.research.google.com/github/tarepan/Scyclone-PyTorch/blob/main/Scyclone_PyTorch.ipynb

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!cat /usr/local/cuda/version.txt

## Setup

Activate notebook intermittently for long session (RUN once **by hand**)
```javascript
const refresher = setInterval(()=>{document.querySelector("colab-connect-button").click();console.log("clicked for long session");}, 1000*60*10);
```

Install package from `tarepan/Scyclone-PyTorch` public repository

In [ ]:
# GoogleDrive
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Phase04/Scyclone-PyTorch/scyclonepytorch')
!unzip -o -q -d /home/ "/content/drive/My Drive/Resources/ScycloneDataset.zip"


In [ ]:
!nvidia-smi

In [ ]:
!mv /home/ScycloneDataset/* /home/
!rm -d /home/ScycloneDataset
!ls /home/

In [ ]:
!ls

## Training

### GPUを使う場合

In [ ]:
!pip install pytorch-lightning munch soundfile torchaudio==0.7.0

In [ ]:
%%writefile Configs/base_gpu.yml

log_dir: "logs/test"
seed: 7354
pretrained_model: ""
batch_size: 64
max_epochs: 10000
ckpt_period: 25
val_period: 10
use_amp: false
device: "gpu"
n_gpus: 1
n_tpus: 0

use_profiler: false

dataset:
  load_from_disk: true
  training_list: "filelists/jsut_jsss_train.txt"
  validation_list: "filelists/jsut_jsss_val.txt"
  n_bins: 128
  hop_length: 128
  sr: 24000
  num_workers: 2
  pin_memory: false



model:
  # trunk_channels: 256
  # n_resblk_g: 7

  # dim_in_d: 128
  noise_sigma_d: 0.01
  # n_resblk_d: 6

critic:
  hinge_offset: 0.5
  lambda_cyc: 10
  lambda_id: 1

optimizer:
  lr: 2.0e-4

In [ ]:
%%capture
!python train.py -p "Configs/base_gpu.yml" -r "logs/test/default/version_6/checkpoints/last.ckpt"

### TPUを使う場合

In [ ]:
!pip install pytorch-lightning munch soundfile npvcc2016
!pip install torchaudio==0.7.0

In [ ]:
VERSION = "1.7"  #@param ["1.5" , "1.6", "1.7", "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
!python train.py -p "Configs/base_tpu.yml"

In [ ]:
!ls "gdrive/My Drive/ML_results/Scyclone"

## Training Optimization

### whole

In [ ]:
# num_worker x pinmemory

!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=0
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=1
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=2
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=4

!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=0 --no_pin_memory
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=1 --no_pin_memory
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=2 --no_pin_memory
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=4 --no_pin_memory

### num_worker

In [ ]:
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=0
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=1
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=2
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=4

### pin_memory

In [ ]:
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=2
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=2 --no_pin_memory

### Profiling

In [ ]:
!python -m scyclonepytorch.Scyclone_main --profiler --max_epochs=5 --num_workers=2  --no_pin_memory # profile mode